In [15]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import timeseries_dataset_from_array
import matplotlib.pyplot as plt
import seaborn as sns
from MultiSeriesWindowsGenerator import MultiSeriesWindowsGenerator
import IPython.display

2023-04-16 10:28:11.343218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
pd.set_option("display.max_row", 200)

In [17]:
df = pd.read_csv("data/all_data_aggr_nonan.csv", index_col=0)

In [18]:
df.mood = df.mood.round(0)

In [20]:
LABELS = ['mood']
REGRESSORS = ['weekday', 'circumplex.arousal', 'circumplex.valence',
              'activity', 'screen', 'call', 'sms', 'appCat.builtin',
              'appCat.communication', 'appCat.entertainment', 'appCat.finance',
              'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
              'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather']

DATE = 'days'  # always correct
IN_STEPS = 7  # use 7 days
OUT_STEPS = 1  # to predict 1 day in the future
GROUPBY = ['subject_id']
BATCH_SIZE = 8

In [21]:
n = len(df)
train_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(
    lambda x: x.iloc[:int(len(x) * 0.7)]).reset_index(drop=True)
val_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(
    lambda x: x.iloc[int(len(x) * 0.7):int(len(x) * 0.9)]).reset_index(drop=True)
test_series = df.groupby(GROUPBY, as_index=False, group_keys=False).apply(
    lambda x: x.iloc[int(len(x) * 0.9):]).reset_index(drop=True)

test_window = MultiSeriesWindowsGenerator(
    input_width=IN_STEPS, label_width=OUT_STEPS, shift=1, batch_size=BATCH_SIZE, GROUPBY=GROUPBY,
    label_columns=LABELS, regressor_columns=REGRESSORS, DATE=DATE, LABELS=LABELS)

test_window.update_datasets(train_series, val_series, test_series)

2023-04-16 10:28:20.089151: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
a, b = test_window.example


In [23]:
lstm_model_classifier = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=11, activation='softmax')
])

MAX_EPOCHS = 5

In [45]:
def compile_and_fit_classifier(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')

    # model.compile(loss=tf.keras.losses.MeanSquaredError(),
    #               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),
    #               metrics=[tf.keras.metrics.MeanAbsoluteError()])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping])
    return history

In [46]:
history = compile_and_fit_classifier(lstm_model_classifier, test_window)
val_performance = {}
performance = {}

probs = lstm_model_classifier.predict(test_window.val)
preds = probs.argmax(axis=-1)
val_performance['LSTM'] = lstm_model_classifier.evaluate(test_window.val, verbose=0)

probs = lstm_model_classifier.predict(test_window.test)
preds = probs.argmax(axis=-1)
performance['LSTM'] = lstm_model_classifier.evaluate(test_window.test, verbose=0)

Epoch 1/5
105/105 [==============================] - 20s 70ms/step - loss: 0.8796 - sparse_categorical_accuracy: 0.6535 - mean_squared_error: 35.8915 - mean_absolute_error: 5.1283 - val_loss: 1.4782 - val_sparse_categorical_accuracy: 0.6243 - val_mean_squared_error: 18.1460 - val_mean_absolute_error: 2.6558
Epoch 2/5
105/105 [==============================] - 6s 42ms/step - loss: 0.8402 - sparse_categorical_accuracy: 0.6762 - mean_squared_error: 35.8249 - mean_absolute_error: 5.1185 - val_loss: 1.4955 - val_sparse_categorical_accuracy: 0.6367 - val_mean_squared_error: 18.1438 - val_mean_absolute_error: 2.6555
Epoch 3/5
68/68 [==============================] - 2s 5ms/step


In [26]:
print("The accuracy on the test set is:",round(performance["LSTM"][1], 4))

The accuracy on the test set is: 0.7704


Construct CI for performance

In [47]:
performance

{'LSTM': [1.2193348407745361,
  0.7314814925193787,
  10.892372131347656,
  1.5919122695922852]}

In [48]:
n_simulations = 10
performances = np.empty((n_simulations,2))
for i in range(n_simulations):
    history = compile_and_fit_classifier(lstm_model_classifier, test_window)
    probs = lstm_model_classifier.predict(test_window.test)
    preds = probs.argmax(axis=-1)
    performances[i,:] = lstm_model_classifier.evaluate(test_window.test, verbose=0)

Epoch 1/5
105/105 [==============================] - 19s 105ms/step - loss: 0.8683 - sparse_categorical_accuracy: 0.6440 - mean_squared_error: 35.8378 - mean_absolute_error: 5.1220 - val_loss: 1.5203 - val_sparse_categorical_accuracy: 0.6314 - val_mean_squared_error: 18.1556 - val_mean_absolute_error: 2.6572
Epoch 2/5
105/105 [==============================] - 6s 41ms/step - loss: 0.8644 - sparse_categorical_accuracy: 0.6619 - mean_squared_error: 35.8288 - mean_absolute_error: 5.1213 - val_loss: 1.5077 - val_sparse_categorical_accuracy: 0.6296 - val_mean_squared_error: 18.1470 - val_mean_absolute_error: 2.6558
Epoch 3/5
105/105 [==============================] - 5s 34ms/step - loss: 0.8036 - sparse_categorical_accuracy: 0.6834 - mean_squared_error: 35.8731 - mean_absolute_error: 5.1268 - val_loss: 1.4956 - val_sparse_categorical_accuracy: 0.6349 - val_mean_squared_error: 18.1275 - val_mean_absolute_error: 2.6535
Epoch 4/5
105/105 [==============================] - 5s 31ms/step - loss: 

In [49]:
accuracies = performances[:,1]